# AI Wine Recommendation Applet

**Date:** 23rd August 2024

## Overview

The AI Wine Recommendation Applet is designed to assist users in selecting the ideal wine based on their preferences. The applet allows users to upload a wine list, extracts relevant data and organizes the data and organizes it into a structured pandas DataFrame. A Machine Learning model is then trained on this data to recommend wines based on user input through an interactive quiz.The process is then generalized to coctails

## Goals

1. **Input Processing:** Implement a robust text process to accurately extract wine data from an uploaded xlsx, or csv file.
2. **Data Structuring:** Organize extracted text into a well-defined DataFrame with columns such as wine name, flavor_notes, type, and price.
3. **Model Training:** Develop a machine learning model that can make personalized wine recommendations based on the user’s preferences.
4. **User Interaction:** Design an interactive quiz to capture user preferences and suggest the best wine accordingly.
5. **Scalability and Customization:** Make the quiz instantly adaptable to any well organized winelist. Users are able to upload their list and instantly have access to to the quiz

## Specifications

- **Input:** File of a wine list provided by the user.
- **Output:** A recommended wine based on user preferences, displayed after they complete a user friendly quiz.
- **Libraries Used:** pandas, scikit-learn, openpyxl, ipywidgets, pydantic.
- **Offline Functionality:** The applet is designed to function entirely offline, relying on locally installed packages and data.


# **<span style="">Prompt 1: Project Overview and Initial Task</span>**

## **<span style="color:blue">I want to create an AI wine recommendation system. It should take a wine list as input, and use its data to train a model for recommending wines based on a users preferences. Can you help me build this step by step? Here is a sample wine list</span>**


In [1]:
pip install pandas scikit-learn openpyxl ipywidgets pydantic


Note: you may need to restart the kernel to use updated packages.


# Let's make the Machine Learning model that could help users decide on their ideal wine

In [8]:
import pandas as pd

# Load the Excel file
file_path = 'wines_balanced.xlsx'
# Read the data into a pandas DataFrame
wine_df = pd.read_excel(file_path)
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Load and prepare the data
wine_df = pd.read_excel('wines_balanced.xlsx')

# Define the weights for different features
type_weight = 1.5
sweetness_weight = 0.4
flavor_notes_weight = 0.3
body_weight = 0.2
occasion_weight = 0.01
price_range_weight = 1.0

# Initialize encoders and binarizers
le_type = LabelEncoder()
le_sweetness = LabelEncoder()
le_body = LabelEncoder()
le_occasion = LabelEncoder()
le_price_range = LabelEncoder()
mlb = MultiLabelBinarizer()

# Encode categorical features
wine_df['type'] = le_type.fit_transform(wine_df['type'])
wine_df['sweetness'] = le_sweetness.fit_transform(wine_df['sweetness'])
wine_df['body'] = le_body.fit_transform(wine_df['body'])
wine_df['occasion'] = le_occasion.fit_transform(wine_df['occasion'])
wine_df['price_range'] = le_price_range.fit_transform(wine_df['price_range'])
wine_df['flavor_notes'] = mlb.fit_transform(wine_df['flavor_notes'].apply(eval)).tolist()

# Prepare the feature matrix X and labels y
flavor_notes_df = pd.DataFrame(wine_df['flavor_notes'].tolist(), index=wine_df.index)
X = pd.concat([
    wine_df[['type', 'sweetness', 'body', 'occasion', 'price_range']],
    flavor_notes_df
], axis=1)

# Convert all column names to strings
X.columns = X.columns.astype(str)

y = wine_df['wine_id']

# Train the NearestNeighbors model
number_of_neighbors = 5
model = NearestNeighbors(n_neighbors=number_of_neighbors, algorithm='auto')
model.fit(X)

print("Model trained successfully.")

# Function to decode one-hot encoded flavor notes back to strings
def decode_one_hot_flavor_notes(one_hot_notes):
    return [label for note, label in zip(one_hot_notes, mlb.classes_) if note == 1]

# Function to make wine recommendations
def make_proposal(qa: dict):
    global model, X, y
    
    # Encode user preferences
    encoded_preferences = [
        le_type.transform([qa['type']])[0] * type_weight,
        le_sweetness.transform([qa['sweetness']])[0] * sweetness_weight,
        le_body.transform([qa['body']])[0] * body_weight,
        le_occasion.transform([qa['occasion']])[0] * occasion_weight,
        le_price_range.transform([qa['price_range']])[0] * price_range_weight
    ]
    
    flavor_note_binarized = mlb.transform([qa['flavor_notes']])[0] * flavor_notes_weight
    encoded_preferences.extend(flavor_note_binarized)
    
    print("Encoded user preferences:", encoded_preferences)  # Debugging

    # Find the nearest neighbors
    _, indices = model.kneighbors([encoded_preferences])

    print("Recommended wine IDs:", indices)  # Debugging

    # Get the recommended wine IDs
    recommended_wine_ids = y.iloc[indices[0]].values

    # Retrieve the corresponding wines
    recommended_wines = wine_df[wine_df['wine_id'].isin(recommended_wine_ids)].copy()
    
    # Mapping encoded values back to strings
    recommended_wines['type'] = le_type.inverse_transform(recommended_wines['type'])
    recommended_wines['sweetness'] = le_sweetness.inverse_transform(recommended_wines['sweetness'])
    recommended_wines['body'] = le_body.inverse_transform(recommended_wines['body'])
    recommended_wines['occasion'] = le_occasion.inverse_transform(recommended_wines['occasion'])
    recommended_wines['price_range'] = le_price_range.inverse_transform(recommended_wines['price_range'])

    # Decode one-hot encoded flavor notes back to strings
    recommended_wines['flavor_notes'] = recommended_wines['flavor_notes'].apply(decode_one_hot_flavor_notes)

    # Format data, return dict object
    res = {}
    for _, row in recommended_wines.iterrows():
        res[row['name']] = {
            "type": row['type'],
            "sweetness": row['sweetness'],
            "flavor_notes": row['flavor_notes'],  # Now properly decoded
            "body": row['body'],
            "occasion": row['occasion'],
            "price_range": row['price_range'],
            "actual_price": row['actual_price'],
            "description": row['description'],
        }

    return res


Model trained successfully.


# Sample usage

In [9]:
def print_recommendations(recommendations):
    print("Your Wine Recommendations:\n")
    for name, details in recommendations.items():
        # Decode flavor notes if they are integers
        flavor_notes = details['flavor_notes']
        if isinstance(flavor_notes[0], int):  # Check if the first element is an integer
            flavor_notes = decode_flavor_notes(flavor_notes)
            
        # Display the recommendations
        print(f"Wine: {name}")
        print(f"  Type: {details['type']}")
        print(f"  Sweetness: {details['sweetness']}")
        print(f"  Description: {details['description']}\n")
        print(f"  Actual Price: ${details['actual_price']}")
        print("-" * 40)

# Example usage
example_preferences = {
    'type': 'Red',
    'sweetness': 'Dry',
    'flavor_notes': ['fruit', 'spice'],
    'body': 'Light',
    'occasion': 'Formal',
    'price_range': '$$$'
}


recommendations = make_proposal(example_preferences)
print_recommendations(recommendations)


Encoded user preferences: [0.0, 0.0, 0.2, 0.01, 2.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.3]
Recommended wine IDs: [[ 0  7 20  9  4]]
Your Wine Recommendations:

Wine: Cabernet Sauvignon
  Type: Red
  Sweetness: Dry
  Description: Cabernet Sauvignon, a dry red with full body and rich notes of fruit and spice. Perfect for formal occasions, enjoyed best from French vineyards.

  Actual Price: $50
----------------------------------------
Wine: Pinot Noir
  Type: Red
  Sweetness: Dry
  Description: Pinot Noir, a dry red with light body and earthy fruit flavors. A choice pick for formal events, especially from Burgundy.

  Actual Price: $45
----------------------------------------
Wine: Zinfandel
  Type: Red
  Sweetness: Dry
  Description: Zinfandel, a dry and full-bodied red wine with vibrant fruity and spicy notes. Perfect for casual get-togethers.

  Actual Price: $18
----------------------------------------
Wine: Malbec
  Type: Red
  Sweetness: Dry
  Description: Malbec, a full-bodied dry red wit

/Users/siskos/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


# Model successfully trained and predicted sample data

# **<span style="">Prompt 2: Interactive Quiz</span>**

## **<span style="color:blue">This looks good, but too complicated. I'd like it to be in the form of an interractive quiz, where I can select the type of wine I want and receive the recommendations</span>**


In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the widgets
type_widget = widgets.Dropdown(options=['White', 'Red'], description='Type:')
sweetness_widget = widgets.Dropdown(options=['Dry', 'Off-dry', 'Sweet'], description='Sweetness:')
body_widget = widgets.Dropdown(options=['Full', 'Medium', 'Light'], description='Body:')
occasion_widget = widgets.Dropdown(options=['Formal', 'Casual'], description='Occasion:')
price_range_widget = widgets.Dropdown(options=['$', '$$', '$$$'], description='Price Range:')
submit_button = widgets.Button(description='Submit')

# Flavor Notes as a series of checkboxes
flavor_notes_widget = widgets.VBox([
    widgets.Checkbox(value=False, description='Fruit'),
    widgets.Checkbox(value=False, description='Spice'),
    widgets.Checkbox(value=False, description='Oak'),
    widgets.Checkbox(value=False, description='Floral'),
    widgets.Checkbox(value=False, description='Herbaceous'),
    widgets.Checkbox(value=False, description='Earthy')
])

# Create an Output widget to display the results
output = widgets.Output()

# Define the function to handle the input
def on_submit(b):
    # Collect selected flavor notes
    selected_flavor_notes = [checkbox.description for checkbox in flavor_notes_widget.children if checkbox.value]
    
    user_preferences = {
        'type': type_widget.value,
        'sweetness': sweetness_widget.value,
        'flavor_notes': selected_flavor_notes,  # Store selected checkboxes as a list
        'body': body_widget.value,
        'occasion': occasion_widget.value,
        'price_range': price_range_widget.value
    }

    # Call the recommendation function
    recommendations = make_proposal(user_preferences)
    
    # Clear previous output and display the new recommendations
    with output:
        clear_output()
        print("Here are your wine recommendations:")
        for name, details in recommendations.items():
            print(f"\n{name}:")
            print(f"  Type: {details.get('type', 'N/A')}")
            print(f"  Sweetness: {details.get('sweetness', 'N/A')}")
            print(f"  Flavor Notes: {', '.join(details.get('flavor_notes', []))}")
            print(f"  Body: {details.get('body', 'N/A')}")
            print(f"  Best Served: {details.get('occasion', 'N/A')}")
            print(f"  Price Range: {details.get('price_range', 'N/A')}")
            print(f"  Description: {details.get('description', 'N/A')}")

# Connect the button to the function
submit_button.on_click(on_submit)

# Display the widgets
display(type_widget, sweetness_widget, flavor_notes_widget, body_widget, occasion_widget, price_range_widget, submit_button, output)


Dropdown(description='Type:', options=('White', 'Red'), value='White')

Dropdown(description='Sweetness:', options=('Dry', 'Off-dry', 'Sweet'), value='Dry')

Dropdown(description='Body:', options=('Full', 'Medium', 'Light'), value='Full')

Dropdown(description='Occasion:', options=('Formal', 'Casual'), value='Formal')

Dropdown(description='Price Range:', options=('$', '$$', '$$$'), value='$')

Button(description='Submit', style=ButtonStyle())

Output()

# **<span style="">Prompt 3: Upload any Winelist</span>**

## **<span style="color:blue">Can you make it work for any wine-list? I want to be able to upload the wine-list, select my preferences, and get instant recommendations</span>**


In [11]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tqdm.notebook import tqdm
import io

# Global variables for the model and encoders
model = None
le_type = LabelEncoder()
le_sweetness = LabelEncoder()
le_body = LabelEncoder()
le_occasion = LabelEncoder()  # Renamed from le_best_served to le_occasion
le_price_range = LabelEncoder()
mlb = MultiLabelBinarizer()

# Define widgets for file upload
upload_widget = widgets.FileUpload(accept='.csv, .xlsx', multiple=False)
train_button = widgets.Button(description='Train Model')
output = widgets.Output()

# Define widgets for wine recommendation inputs
type_widget = widgets.Dropdown(options=[], description='Type:')
sweetness_widget = widgets.Dropdown(options=[], description='Sweetness:')
body_widget = widgets.Dropdown(options=[], description='Body:')
occasion_widget = widgets.Dropdown(options=[], description='Occasion:')  # Renamed from best_served_widget to occasion_widget
price_range_widget = widgets.Dropdown(options=[], description='Price Range:')

# Replace the SelectMultiple with a series of checkboxes for flavor notes
flavor_notes_widget = widgets.VBox([
    widgets.Checkbox(value=False, description='Fruit'),
    widgets.Checkbox(value=False, description='Spice'),
    widgets.Checkbox(value=False, description='Oak'),
    widgets.Checkbox(value=False, description='Floral'),
    widgets.Checkbox(value=False, description='Herbaceous'),
    widgets.Checkbox(value=False, description='Earthy')
])

submit_button = widgets.Button(description='Submit')

# Display results
recommendation_output = widgets.Output()

# Function to handle file upload and training with progress bar
def train_model(file_content, file_name):
    global model, wine_df

    with output:
        clear_output()
        print("Starting the model training process...")
        
        # Step 1: Load the data
        print("Step 1/4: Loading the data...")
        try:
            file_extension = file_name.split('.')[-1]
            if file_extension == 'xlsx':
                wine_df = pd.read_excel(io.BytesIO(file_content))
            elif file_extension == 'csv':
                wine_df = pd.read_csv(io.BytesIO(file_content))
            else:
                print("Unsupported file format.")
                return
            print("Data loaded successfully.")
        except Exception as e:
            print(f"Error loading data: {e}")
            return
        
        # Step 2: Preprocessing
        print("Step 2/4: Preprocessing the data...")
        try:
            wine_df['type'] = le_type.fit_transform(wine_df['type'])
            wine_df['sweetness'] = le_sweetness.fit_transform(wine_df['sweetness'])
            wine_df['body'] = le_body.fit_transform(wine_df['body'])
            wine_df['occasion'] = le_occasion.fit_transform(wine_df['occasion'])  # Updated column name
            wine_df['price_range'] = le_price_range.fit_transform(wine_df['price_range'])
            wine_df['flavor_notes'] = mlb.fit_transform(wine_df['flavor_notes'].apply(eval)).tolist()
            print("Data preprocessing completed.")
        except Exception as e:
            print(f"Error during preprocessing: {e}")
            return
        
        # Step 3: Preparing the feature matrix
        print("Step 3/4: Preparing the feature matrix...")
        try:
            flavor_notes_df = pd.DataFrame(wine_df['flavor_notes'].tolist(), index=wine_df.index)
            X = pd.concat([
                wine_df[['type', 'sweetness', 'body', 'occasion', 'price_range']],  # Updated column name
                flavor_notes_df
            ], axis=1)
            y = wine_df['wine_id']

            # Ensure all column names are strings
            X.columns = X.columns.astype(str)
            
            print("Feature matrix preparation completed.")
        except Exception as e:
            print(f"Error preparing feature matrix: {e}")
            return
        
        # Step 4: Training the model with progress bar
        print("Step 4/4: Training the Nearest Neighbors model...")
        try:
            progress = tqdm(total=100, desc="Training Progress", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]')
            model = NearestNeighbors(n_neighbors=5, algorithm='auto')
            model.fit(X)
            progress.update(100)
            progress.close()
            print("Model training completed successfully.")
        except Exception as e:
            print(f"Error during model training: {e}")
            return
        
        # Update widget options based on the new data
        type_widget.options = list(le_type.classes_)
        sweetness_widget.options = list(le_sweetness.classes_)
        body_widget.options = list(le_body.classes_)
        occasion_widget.options = list(le_occasion.classes_)  # Updated widget
        price_range_widget.options = list(le_price_range.classes_)
        
        print("Widget options updated with new data.")

# Function to make wine recommendations
def make_proposal(qa):
    global model, wine_df

    # Encode user preferences
    encoded_preferences = [
        le_type.transform([qa['type']])[0],
        le_sweetness.transform([qa['sweetness']])[0],
        le_body.transform([qa['body']])[0],
        le_occasion.transform([qa['occasion']])[0],  # Updated key name
        le_price_range.transform([qa['price_range']])[0]
    ]
    
    flavor_note_binarized = mlb.transform([qa['flavor_notes']])[0]
    encoded_preferences.extend(flavor_note_binarized)
    
    # Get the closest matches
    _, indices = model.kneighbors([encoded_preferences])
    recommended_wine_ids = wine_df.iloc[indices[0]]['wine_id'].values
    
    # Filter recommended wines
    recommended_wines = wine_df[wine_df['wine_id'].isin(recommended_wine_ids)].copy()
    
    # Decode the label encoded features
    recommended_wines['type'] = le_type.inverse_transform(recommended_wines['type'])
    recommended_wines['sweetness'] = le_sweetness.inverse_transform(recommended_wines['sweetness'])
    recommended_wines['body'] = le_body.inverse_transform(recommended_wines['body'])
    recommended_wines['occasion'] = le_occasion.inverse_transform(recommended_wines['occasion'])  # Updated column name
    recommended_wines['price_range'] = le_price_range.inverse_transform(recommended_wines['price_range'])
    recommended_wines['flavor_notes'] = recommended_wines['flavor_notes'].apply(
        lambda x: [mlb.classes_[i] for i in range(len(x)) if x[i] == 1])

    # Sort recommendations so that wines of the same type as the user preference appear first
    recommended_wines = recommended_wines.sort_values(by='type', ascending=False, key=lambda x: x == qa['type'])
    
    # Prepare the response dictionary
    res = {}
    for _, row in recommended_wines.iterrows():
        res[row['name']] = {
            "type": row['type'],
            "sweetness": row['sweetness'],
            "flavor_notes": row['flavor_notes'],
            "body": row['body'],
            "occasion": row['occasion'],  # Updated key name
            "price_range": row['price_range'],
            "actual_price": row['actual_price'],
            "description": row['description'],
        }

    return res

# Function to print recommendations
def print_recommendations(recommendations):
    with recommendation_output:
        clear_output()
        print("Your Wine Recommendations:\n")
        for name, details in recommendations.items():
            print(f"Wine: {name}")
            print(f"  Type: {details['type']}")
            print(f"  Sweetness: {details['sweetness']}")
            print(f"  Flavor Notes: {', '.join(details['flavor_notes'])}")
            print(f"  Body: {details['body']}")
            print(f"  Occasion: {details['occasion']}")  # Updated key name
            print(f"  Price Range: {details['price_range']}")
            print(f"  Actual Price: ${details['actual_price']}")
            print(f"  Description: {details['description']}\n")
            print("-" * 40)

# Event handler for the submit button
def on_submit(b):
    # Collect selected flavor notes from checkboxes
    selected_flavor_notes = [checkbox.description for checkbox in flavor_notes_widget.children if checkbox.value]
    
    user_preferences = {
        'type': type_widget.value,
        'sweetness': sweetness_widget.value,
        'flavor_notes': selected_flavor_notes,
        'body': body_widget.value,
        'occasion': occasion_widget.value,  # Updated key name
        'price_range': price_range_widget.value
    }
    recommendations = make_proposal(user_preferences)
    print_recommendations(recommendations)

# Event handler for the train button
def on_train_click(b):
    if len(upload_widget.value) == 0:
        with output:
            clear_output()
            print("Please upload a file before training.")
        return

    uploaded_file = list(upload_widget.value)[0]  # Access the first file
    file_content = uploaded_file['content']
    file_name = uploaded_file['name']
    train_model(file_content, file_name)

# Connect the buttons to the functions
train_button.on_click(on_train_click)
submit_button.on_click(on_submit)

# Display the interface
display(widgets.VBox([upload_widget, train_button, output]))
display(widgets.VBox([type_widget, sweetness_widget, flavor_notes_widget, body_widget, occasion_widget, price_range_widget, submit_button, recommendation_output]))


# **<span style="">Prompt 4: Generalize Recommender system to coctails</span>**

## **<span style="color:blue">My girlfriend wants to order a cocktail instead. Can you make it work for a coctail menu? Don't include the main spirits in the quiz, just the main tastes</span>**


In [12]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder

# Load the cocktail menu
file_path = 'Cocktail_Menu.csv'  # Update this with the correct path
cocktail_df = pd.read_csv(file_path)

# Define ingredient categories
ingredient_categories = {
    "Sweet 🍬": ["Sugar", "Honey", "Simple Syrup", "Agave Syrup"],
    "Spicy 🌶️": ["Bitters", "Cinnamon", "Nutmeg", "Clove", "Tabasco", "Pepper"],
    "Orange 🍊": ["Orange Juice", "Triple Sec", "Orange Peel", "Grand Marnier"],
    "Lemon 🍋": ["Lemon Juice", "Limoncello", "Lemon Peel"],
    "Lime 🍈": ["Lime Juice", "Lime Peel"],
    "Herbal 🌿": ["Mint", "Basil", "Rosemary", "Thyme"],
    "Other 🧂": ["Salt", "Soda Water", "Grenadine"]
}


# Function to map ingredients to categories
def map_ingredients_to_categories(ingredients):
    mapped = set()
    for ingredient in ingredients:
        for category, items in ingredient_categories.items():
            if ingredient in items:
                mapped.add(category)
    return list(mapped)

# Apply the mapping to the dataset
cocktail_df['Ingredient Categories'] = cocktail_df['Ingredients'].apply(
    lambda ingredients: map_ingredients_to_categories([i.strip() for i in ingredients.split(',')])
)

# Multi-label binarize the ingredient categories
mlb = MultiLabelBinarizer()
categories_encoded = mlb.fit_transform(cocktail_df['Ingredient Categories'])

# One-hot encode the 'Main Spirit' column
one_hot_encoder = OneHotEncoder()
main_spirit_encoded = one_hot_encoder.fit_transform(cocktail_df[['Main Spirit']]).toarray()

# Combine the encoded data
X = pd.DataFrame(main_spirit_encoded, columns=one_hot_encoder.get_feature_names_out(['Main Spirit']))
X = pd.concat([X, pd.DataFrame(categories_encoded, columns=mlb.classes_)], axis=1)

# To emphasize the main spirit, manually repeat its encoding multiple times
main_spirit_weight = 1000  # Increase this weight to make the spirit more important
repeated_spirit_columns = pd.DataFrame(
    X.loc[:, one_hot_encoder.get_feature_names_out(['Main Spirit'])].values.repeat(main_spirit_weight, axis=1),
    columns=[f"{col}_w{i}" for col in one_hot_encoder.get_feature_names_out(['Main Spirit']) for i in range(main_spirit_weight)]
)
X_weighted = pd.concat([repeated_spirit_columns, X.drop(columns=one_hot_encoder.get_feature_names_out(['Main Spirit']))], axis=1)

# Initialize k-NN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_weighted)

# Define widgets for cocktail recommendation inputs
quirky_text = widgets.HTML("<b>Select the flavors you would like to enjoy:</b>")
category_widget = widgets.VBox([
    widgets.Checkbox(value=False, description=category)
    for category in ingredient_categories.keys()
])
submit_button = widgets.Button(description='Find Cocktail')
recommendation_output = widgets.Output()

# Function to make cocktail recommendations
def make_cocktail_recommendation(preferences):
    # Encode the user inputs
    user_categories = mlb.transform([preferences['categories']])[0]

    # Create the input vector for k-NN (main spirit is emphasized internally)
    user_input = list([0] * main_spirit_weight * len(one_hot_encoder.get_feature_names_out(['Main Spirit']))) + list(user_categories)

    # Ensure the user_input matches the weighted feature set
    user_input = pd.DataFrame([user_input])

    # Get nearest neighbors
    distances, indices = knn.kneighbors(user_input)
    recommended_cocktails = cocktail_df.iloc[indices[0]]

    return recommended_cocktails

# Function to display recommendations
def print_cocktail_recommendations(recommendations):
    with recommendation_output:
        clear_output()
        if recommendations.empty:
            print("No cocktails match your preferences.")
        else:
            print("Here are your cocktail recommendations:")
            for _, row in recommendations.iterrows():
                print(f"\nCocktail: {row['Cocktail Name']}")
                print(f"  Main Spirit: {row['Main Spirit']}")
                print(f"  Ingredient Categories: {', '.join(row['Ingredient Categories'])}")

# Event handler for the submit button
def on_submit(b):
    selected_categories = [
        checkbox.description for checkbox in category_widget.children if checkbox.value
    ]
    user_preferences = {
        'categories': selected_categories
    }
    recommendations = make_cocktail_recommendation(user_preferences)
    print_cocktail_recommendations(recommendations)

# Connect the button to the function
submit_button.on_click(on_submit)

# Display the interface
display(quirky_text, category_widget, submit_button, recommendation_output)


HTML(value='<b>Select the flavors you would like to enjoy:</b>')

Button(description='Find Cocktail', style=ButtonStyle())

Output()